# TOBIT REGRESSION ANALYSIS

In [130]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('/content/NSSO68 (1).csv')
df.head()

<ipython-input-114-1e3ebcbb157a>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/NSSO68 (1).csv')


,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,pickle_v,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot
0,1,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2.0,12.000000,154.18
1,2,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2.0,333.000000,484.95
2,3,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2.0,35.000000,214.84
3,4,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2.0,168.333333,302.30
4,5,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2.0,15.000000,148.00


In [ ]:
df.shape

(95999, 384)

In [ ]:
list(df.columns)

['slno',
 'grp',
 'Round_Centre',
 'FSU_number',
 'Round',
 'Schedule_Number',
 'Sample',
 'Sector',
 'state',
 'State_Region',
 'District',
 'Stratum_Number',
 'Sub_Stratum',
 'Schedule_type',
 'Sub_Round',
 'Sub_Sample',
 'FOD_Sub_Region',
 'Hamlet_Group_Sub_Block',
 't',
 'X_Stage_Stratum',
 'HHS_No',
 'Level',
 'Filler',
 'hhdsz',
 'NIC_2008',
 'NCO_2004',
 'HH_type',
 'Religion',
 'Social_Group',
 'Whether_owns_any_land',
 'Type_of_land_owned',
 'Land_Owned',
 'Land_Leased_in',
 'Otherwise_possessed',
 'Land_Leased_out',
 'Land_Total_possessed',
 'During_July_June_Cultivated',
 'During_July_June_Irrigated',
 'NSS',
 'NSC',
 'MLT',
 'land_tt',
 'Cooking_code',
 'Lighting_code',
 'Dwelling_unit_code',
 'Regular_salary_earner',
 'Perform_Ceremony',
 'Meals_seved_to_non_hhld_members',
 'Possess_ration_card',
 'Type_of_ration_card',
 'MPCE_URP',
 'MPCE_MRP',
 'Person_Srl_No',
 'Relation',
 'Sex',
 'Age',
 'Marital_Status',
 'Education',
 'Days_Stayed_away',
 'No_of_Meals_per_day',
 'Me

In [125]:
# Define the features and the response variable
features = ['hhdsz', 'Religion', 'Social_Group', 'Type_of_land_owned', 'Land_Owned',
            'MPCE_URP', 'Age', 'Sex', 'Education', 'Regular_salary_earner']
x=df[features]
y=df['nonvegtotal_q']

In [126]:
# Add a constant to the independent variables
X = sm.add_constant(X)

In [132]:
class TobitModel:
    def __init__(self, y, X, left_censoring=0):
        self.y = y
        self.X = sm.add_constant(X)  # Add constant term for intercept
        self.left_censoring = left_censoring

    def log_likelihood(self, params):
        beta = params[:-1]
        sigma = params[-1]
        xb = np.dot(self.X, beta)
        ll = (self.y > self.left_censoring) * (np.log(norm.pdf((self.y - xb) / sigma) / sigma)) + \
             (self.y <= self.left_censoring) * (np.log(norm.cdf((self.left_censoring - xb) / sigma)))
        return -np.sum(ll)

    def fit(self):
        init_params = np.append(np.zeros(self.X.shape[1]), 1)  # Initial guess for beta and sigma
        bounds = [(None, None)] * (self.X.shape[1]) + [(1e-5, None)]  # Bounds for optimization
        result = minimize(self.log_likelihood, init_params, bounds=bounds, method='L-BFGS-B')
        self.params = result.x
        return self

    def summary(self):
        beta = self.params[:-1]
        sigma = self.params[-1]
        return beta, sigma

# Prepare the data
y = df['nonvegtotal_q']
X = df[features]

# Fit the model
tobit_model = TobitModel(y, X)
tobit_model.fit()
beta, sigma = tobit_model.summary()

print("Coefficients:", beta)
print("Sigma:", sigma)


<ipython-input-132-87fd0e692e80>:11: RuntimeWarning: divide by zero encountered in log
  ll = (self.y > self.left_censoring) * (np.log(norm.pdf((self.y - xb) / sigma) / sigma)) + \


Coefficients: [-1.61892244e-04 -7.69008971e-04 -2.34206491e-04 -7.00620822e-04
 -2.44158282e-04 -1.12467871e-01 -3.13455080e-01 -7.74402750e-03
 -1.79795181e-04 -1.02167250e-03 -2.78968778e-04]
Sigma: 1.0
